# 翻译 Relay 程序为 Relax

In [1]:
import numpy as np
import tvm
from tvm.relay import testing
from tvm import relax, relay
from tvm.relax.testing import relay_translator, nn
from tvm.runtime import vm as vm_rt
from tvm.script import relax as R

In [2]:
relay_mod, _ = testing.resnet.get_workload(num_layers=50, batch_size=1, dtype="float32")
# translate the ResNet model from Relay to Relax
target = tvm.target.Target("llvm", host="llvm")
relax_mod = relay_translator.from_relay(relay_mod["main"], target)

# print the ResNet IRmodule got translated
relax_mod.show()

# build the IRModule and create relax vm
ex = relax.build(relax_mod, target)
vm = relax.VirtualMachine(ex, tvm.cpu())

# init weights and run the model on relax vm
shape = (1, 3, 224, 224)
data = tvm.nd.array(np.random.rand(*shape).astype(np.float32))
params = nn.init_params(relax_mod)
res = vm["main"](data, *params)

# check correctness by comparing with relay result
exe = relay.vm.compile(relay_mod, target)
relay_vm = vm_rt.VirtualMachine(exe, tvm.cpu())
inputs = [data] + params
expected_output = relay_vm.run(*inputs)
np.testing.assert_allclose(res.numpy(), expected_output.numpy(), rtol=1e-4, atol=1e-4)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
